In [1]:
import functools
import numpy as np
import tensorflow as tf
from keras.datasets import cifar100
from keras import Sequential
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.optimizers import SGD
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, LeakyReLU
from keras.layers.normalization import BatchNormalization
from tensorflow.keras.utils import to_categorical
from keras.utils.generic_utils import get_custom_objects
from keras.initializers import RandomNormal, GlorotNormal

In [2]:
(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=.2)

In [3]:
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

In [4]:
x_train.shape, y_train.shape, x_val.shape, y_val.shape, x_test.shape, y_test.shape

((40000, 32, 32, 3),
 (40000, 100),
 (10000, 32, 32, 3),
 (10000, 100),
 (10000, 32, 32, 3),
 (10000, 100))

In [5]:
train_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)
val_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)
test_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)

train_generator.fit(x_train)
val_generator.fit(x_val)
test_generator.fit(x_test)

In [6]:
def simplenet(act, s = 2):
    model = Sequential()

    model.add(Conv2D(64, (3,3), padding='same', kernel_initializer=GlorotNormal(), input_shape=(32, 32, 3)))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(128, (3,3), padding='same', kernel_initializer=GlorotNormal()))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(128, (3,3), padding='same', kernel_initializer=RandomNormal(stddev=0.01)))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(128, (3,3), padding='same', kernel_initializer=RandomNormal(stddev=0.01)))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(MaxPooling2D(pool_size=(2,2), strides=s))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(128, (3,3), padding='same', kernel_initializer=RandomNormal(stddev=0.01)))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(128, (3,3), padding='same', kernel_initializer=GlorotNormal()))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(256, (3,3), padding='same', kernel_initializer=GlorotNormal()))
    model.add(MaxPooling2D(pool_size=(2,2), strides=s))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(256, (3,3), padding='same', kernel_initializer=GlorotNormal()))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(256, (3,3), padding='same', kernel_initializer=GlorotNormal()))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    model.add(MaxPooling2D(pool_size=(2,2), strides=s))
    
    model.add(Conv2D(512, (3,3), padding='same', kernel_initializer=GlorotNormal()))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))

    model.add(Conv2D(2048, (1,1), padding='same', kernel_initializer=GlorotNormal()))
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(256, (1,1), padding='same', kernel_initializer=GlorotNormal()))
    model.add(Activation(act))
    model.add(MaxPooling2D(pool_size=(2,2), strides=s))
    model.add(Dropout(0.2))

    model.add(Conv2D(256, (3,3), padding='same', kernel_initializer=GlorotNormal()))
    model.add(Activation(act))
    model.add(MaxPooling2D(pool_size=(2,2), strides=s))

    model.add(Flatten())
    model.add(Dense(100, activation='softmax'))

    return model

In [9]:
def gelu(x):
    return 0.5 * x * (1 + tf.tanh(tf.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3))))
get_custom_objects().update({'gelu': Activation(gelu)})

def swish(x):
    return x * tf.sigmoid(x)
get_custom_objects().update({'swish': Activation(swish)})

get_custom_objects().update({'leaky-relu': Activation(LeakyReLU(alpha=0.2))})

# act_func = ['tanh', 'relu', 'leaky-relu', 'elu', 'selu', 'gelu', 'swish']

model = simplenet('relu')

batch_size = 128
epochs = 50

top3_acc = functools.partial(tf.keras.metrics.top_k_categorical_accuracy, k=3)

top3_acc.__name__ = 'top3_acc'

model.compile(optimizer=SGD(), loss='categorical_crossentropy', metrics=['accuracy', top3_acc, 'top_k_categorical_accuracy'])

history = model.fit(train_generator.flow(x_train, y_train, batch_size=batch_size), epochs=epochs,
                      validation_data=val_generator.flow(x_val, y_val, batch_size=batch_size), verbose=1)

Epoch 1/50
313/313 [==============================] - 45s 131ms/step - loss: 4.5898 - accuracy: 0.0151 - top3_acc: 0.0429 - top_k_categorical_accuracy: 0.0681 - val_loss: 4.6125 - val_accuracy: 0.0120 - val_top3_acc: 0.0322 - val_top_k_categorical_accuracy: 0.0528
Epoch 2/50
313/313 [==============================] - 40s 128ms/step - loss: 4.4319 - accuracy: 0.0275 - top3_acc: 0.0751 - top_k_categorical_accuracy: 0.1178 - val_loss: 4.6967 - val_accuracy: 0.0096 - val_top3_acc: 0.0295 - val_top_k_categorical_accuracy: 0.0505
Epoch 3/50
313/313 [==============================] - 41s 131ms/step - loss: 4.2857 - accuracy: 0.0462 - top3_acc: 0.1123 - top_k_categorical_accuracy: 0.1647 - val_loss: 5.2599 - val_accuracy: 0.0091 - val_top3_acc: 0.0284 - val_top_k_categorical_accuracy: 0.0507
Epoch 4/50
313/313 [==============================] - 41s 130ms/step - loss: 4.1041 - accuracy: 0.0649 - top3_acc: 0.1570 - top_k_categorical_accuracy: 0.2271 - val_loss: 5.9970 - val_accuracy: 0.0095 - va

In [10]:
print(history.history)
# y_pred = np.argmax(model.predict(x_test), axis=1)
# y_true = np.argmax(y_test,axis=1)

# print(y_pred.shape)
# print(y_true.shape)

# print(np.sum(y_pred == y_true) / y_pred.shape[0])

print(model.evaluate(x_test, y_test))

{'loss': [4.531716823577881, 4.401214122772217, 4.246602535247803, 4.055988311767578, 3.879645347595215, 3.7295753955841064, 3.5972793102264404, 3.471210479736328, 3.350360870361328, 3.226804256439209, 3.116773843765259, 3.0133934020996094, 2.9047181606292725, 2.8115971088409424, 2.7179524898529053, 2.6535143852233887, 2.5763585567474365, 2.512035846710205, 2.448869228363037, 2.377668857574463, 2.324556350708008, 2.275979518890381, 2.222717761993408, 2.172999382019043, 2.1344196796417236, 2.0940561294555664, 2.0501205921173096, 2.0017576217651367, 1.9736148118972778, 1.9381152391433716, 1.904128909111023, 1.8738411664962769, 1.8395894765853882, 1.804709792137146, 1.77095365524292, 1.7463737726211548, 1.7247978448867798, 1.700275182723999, 1.668502688407898, 1.644116759300232, 1.6212711334228516, 1.6045671701431274, 1.5778793096542358, 1.5551838874816895, 1.5282397270202637, 1.5158483982086182, 1.4930490255355835, 1.47233247756958, 1.4620229005813599, 1.437757134437561], 'accuracy': [0.